In [86]:
import math
import numpy as np
import pandas as pd
import spacy
import time
import logging

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

### 1-) Data and Tokenization

In [87]:
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [88]:
len(text)

1115394

In [89]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {char: idx for idx, char in enumerate(chars)}
itos = {idx: char for idx, char in enumerate(chars)}

def encoder(txt):
    return [stoi[char] for char in txt]

def decoder(idx):
    return "".join([itos[i] for i in idx])


In [90]:
print(encoder('hii there'))
print(decoder(encoder('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [91]:
data = torch.tensor(encoder(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [92]:
decoder(data[:1000].tolist())

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

### 2-) Train and test data

In [93]:
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [94]:
block_size = 8
train_data[:block_size +1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [95]:
x = train_data[:block_size]
y = train_data[1:block_size +1]
print(x,y)

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is: {context}, target is: {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])
When input is: tensor([18]), target is: 47
When input is: tensor([18, 47]), target is: 56
When input is: tensor([18, 47, 56]), target is: 57
When input is: tensor([18, 47, 56, 57]), target is: 58
When input is: tensor([18, 47, 56, 57, 58]), target is: 1
When input is: tensor([18, 47, 56, 57, 58,  1]), target is: 15
When input is: tensor([18, 47, 56, 57, 58,  1, 15]), target is: 47
When input is: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is: 58


### 3-) Get Batch

In [96]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences to process at once
block_size = 8 # What is the maximum length of sequences in a batch

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # -block_size avoid going over the end of the data
    print(f"Index: {ix}")
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
xb,yb = get_batch('train')

print("Inputs:")
print(xb.shape)
print(xb)
print("Outputs:")
print(yb.shape)
print(yb)
print("------------------------------------------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b, t]
        print(f"When input is: {context.tolist()}, target is: {target}")

Index: tensor([ 76049, 234249, 934904, 560986])
Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Outputs:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------------------------
When input is: [24], target is: 43
When input is: [24, 43], target is: 58
When input is: [24, 43, 58], target is: 5
When input is: [24, 43, 58, 5], target is: 57
When input is: [24, 43, 58, 5, 57], target is: 1
When input is: [24, 43, 58, 5, 57, 1], target is: 46
When input is: [24, 43, 58, 5, 57, 1, 46], target is: 43
When input is: [24, 43, 58, 5, 57, 1, 46, 43], target is: 39
When input is: [44], target is: 53
When input is: [44, 53], target is: 56
When input is: [44, 53, 56], target is: 1
When input i

In [97]:
print(data[76049:76049+8])
print(data[234249:234249+8])
print(data[934904:934904+8])

tensor([24, 43, 58,  5, 57,  1, 46, 43])
tensor([44, 53, 56,  1, 58, 46, 39, 58])
tensor([52, 58,  1, 58, 46, 39, 58,  1])


### 4-) Create Bigram class

In [102]:
torch.manual_seed(1337)
batch_size = 4  # How many independent sequences to process at once
block_size = 8  # What is the maximum length of sequences in a batch

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # this is 65x65 matrix. xb input of first matrix [24, 43, 58,  5, 57,  1, 46, 43]. 24 goes 24th row, 43 goes 43th row etc.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # Batch, Time, Channel (B,T,C),(4, 8, 65)
        
        if targets is None:
            loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Cross Entropy expects logits to be of shape (B*T, C) and targets to be of shape (B*T)
            targets = targets.view(B*T)  # Cross Entropy expects logits to be of shape (B*T, C) and targets to be of shape (B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current sequence
        for _ in range(max_new_tokens):
            # get prediction
            logits, loss = self(idx)
            # focus only last token
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # smaple from probabilities to get indices
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            return torch.cat((idx, idx_next), dim=1) # (B, T+1)
    

In [103]:
# keep in mind that xb and yb are 4x8 matrices which we created in the previous steps (get_batch function)
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(vocab_size)
print(logits.shape)
#print(logits)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decoder(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))

65
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

S
